<a href="https://www.kaggle.com/code/aleksandrmorozov123/cv-object-detection?scriptVersionId=198418658" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/coronal_t1wce_2_class.yaml
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/55 (2).txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/40.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/44.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/36.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/47.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/19.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/45.txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/57 (2).txt
/kaggle/input/brain-tumor-object-detection-datasets/coronal_t1wce_2_class/labels/test/59.txt
/kaggle/input/brain-tumor-object-detection-datasets/co

**Crowd counting**

In [2]:
import torch
import os
import h5py
from scipy import io

if not os.path.exists('CSRNet-pytorch/'):
    %pip install -U scipy torch_snippets torch_summary
    !git clone https://github.com/sizhky/CSRNet-pytorch.git

%cd CSRNet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.8 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26306 sha256=77be98fde25d24f3466f6c990f154e7c52839f5be2995e2bd96a8ec6533e14c5
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing
Note: you may need to restart the kernel to use updated packages.
Cloning into 'CSRNet-pytorch'...
remote: Enum

In [3]:
from torch_snippets import *
from torchvision import transforms

In [4]:
part_A = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/'
image_folder = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/images/'

heatma_folder = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/ground-truth-h5/'
gt_folder = '/kaggle/input/shanghaitech-with-people-density-map/ShanghaiTech/part_A/train_data/ground-truth/'

In [5]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]

In [6]:
# define training and validation dataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tfm = transforms.Compose([transforms.ToTensor()])

class Crowds(Dataset):
    def __init__(self, stems):
        self.stems = stems

    def __len__(self):
        return len(self.stems)

    def __getitem__(self, ix):
        _stem = self.stems[ix]
        image_path = image_folder
        heatmap_path = heatmap_folder
        gt_path = gt_folder

        pts = io.loadmat(gt_path)
        pts = len(pts['image_info'][0,0][0,0][0])

        image = read(image_path, 1)
        with h5py.File(heatmap_path, 'r') as hf:
            gt = hf['density'][:]
        gt = resize(gt, 1/8)*64
        return image.copy(), gt.copy(), pts

    def collate_fn(self, batch):
        ims, gts, pts = list(zip(*batch))
        ims = torch.cat([tfm(im)[None] for im in ims]).to(device)
        gts = torch.cat([tfm(gt)[None] for gt in gts]).to(device)
        return ims, gts, torch.tensor(pts).to(device)

    def choose(self):
        return self[randint(len(self))]

from sklearn.model_selection import train_test_split
trn_stems, val_stems = train_test_split(stems(Glob(image_folder)), random_state=10)

trn_ds = Crowds(trn_stems)
val_ds = Crowds(val_stems)

trn_dl = DataLoader(trn_ds, batch_size=1, shuffle=True, collate_fn=trn_ds.collate_fn)
val_dl = DataLoader(val_ds, batch_size=1, shuffle=True, collate_fn=val_ds.collate_fn)

In [7]:
from torchvision import models
from utils import save_net,load_net

def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate, dilation=d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)